<a href="https://colab.research.google.com/github/selhabti/Test-Data-Engineer/blob/main/Test_Data_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the required modules
import pandas as pd
from geopy.geocoders import Nominatim
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format','{:.7f}'.format)
#Creating an instance of Nominatim Class
geolocator = Nominatim(user_agent="TestDataEngineer")

# Importing the csv file
df_address = pd.read_csv("https://raw.githubusercontent.com/selhabti/Test-Data-Engineer/main/Adress.csv")

# Explorating the dataframe:
<hr>

In [ ]:
df_address

In [ ]:
df_address.isnull().sum().sum()

Add a new column with the complete addresses:

In [ ]:
df_address['address_comp'] = df_address[['address', 'city']].agg(', '.join, axis=1)
df_address.head()

Apply the $Geocode$ function to obtain the longitude and latitude:

In [ ]:
df_address['location'] = df_address['address_comp'].apply(lambda x: geolocator.geocode(x,timeout=10000))
df_address['Lat'] = df_address['location'].apply(lambda x: x.latitude if x else None)
df_address['Lon'] = df_address['location'].apply(lambda x: x.longitude if x else None)
df_address.head()

Handling NAN values:

In [ ]:
df_address.isnull().sum()

In [ ]:
df_NAN = df_address[df_address['location'].isnull()]
df_NAN.head()

In [ ]:
df_NAN['location'] = df_NAN['city'].apply(lambda x: geolocator.geocode(x,timeout=10000))
df_NAN['Lat'] = df_NAN['location'].apply(lambda x: x.latitude if x else None)
df_NAN['Lon'] = df_NAN['location'].apply(lambda x: x.longitude if x else None)
df_NAN.head()

In [ ]:
df_withoutNAN = df_address.dropna()

In [ ]:
df_withoutNAN

In [ ]:
df_merge = pd.concat([df_withoutNAN,df_NAN]).sort_values(by=['address_id'])
df_merge

In [ ]:
df_geo = df_merge.drop(df_merge.columns[[4,5]], axis=1)  

In [ ]:
df_geo

In [ ]:
df_geo.dtypes

## Exporting the csv file:

In [ ]:
from google.colab import files
df_geo.to_csv('table_add_update.csv',float_format="%.7f") 
files.download('table_add_update.csv')